In [ ]:
!git clone https://github.com/mogomaa79/passport_enhance

Cloning into 'passport_enhance'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 23 (delta 3), reused 19 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (23/23), 180.17 KiB | 22.52 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [ ]:
!pip install -r /content/passport_enhance/requirements.txt

  Cloning https://github.com/XPixelGroup/BasicSR (to revision master) to /tmp/pip-req-build-r0tto7cj
  Running command git clone --filter=blob:none --quiet https://github.com/XPixelGroup/BasicSR /tmp/pip-req-build-r0tto7cj
  Resolved https://github.com/XPixelGroup/BasicSR to commit 8d56e3a045f9fb3e1d8872f92ee4a4f07f886b0a
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none

In [ ]:
!pip uninstall -y numpy
!pip install numpy==1.23.5

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 115.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.24 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.
albumentations 2.0.8 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.23.5 which is incompatible.
pymc 5.23.0 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have numpy 1.23.5 which is incom

In [ ]:
import os
import subprocess
import torch
import cv2
import uuid
import numpy as np
from PIL import Image
from basicsr.archs.srvgg_arch import SRVGGNetCompact
# from gfpgan.utils import GFPGANer
from realesrgan.utils import RealESRGANer

def runcmd(cmd, verbose = False):

    process = subprocess.Popen(
        cmd,
        stdout = subprocess.PIPE,
        stderr = subprocess.PIPE,
        text = True,
        shell = True
    )
    std_out, std_err = process.communicate()
    if verbose:
        print(std_out.strip(), std_err)
    pass

# if not os.path.exists('GFPGANv1.4.pth'):
#     runcmd("wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth -P .")
if not os.path.exists('realesr-general-x4v3.pth'):
    runcmd("wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-general-x4v3.pth -P .")

model = SRVGGNetCompact(num_in_ch=3, num_out_ch=3, num_feat=64, num_conv=32, upscale=4, act_type='prelu')
model_path = 'realesr-general-x4v3.pth'
half = True if torch.cuda.is_available() else False
upsampler = RealESRGANer(scale=2, model_path=model_path, model=model, tile=0, tile_pad=10, pre_pad=0, half=half)


In [ ]:
def enhance_image(
    input_image_path: str,
    scale: int=2,
    enhance_mode: str="Only Image Enhance",
    output_dir='./enhanced_output/'  # Default output directory in Colab
):
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Initialize enhancers based on mode
    # only_face = enhance_mode == "Only Face Enhance"
    # if enhance_mode == "Only Face Enhance":
    #     face_enhancer = GFPGANer(model_path='GFPGANv1.4.pth', upscale=scale, arch='clean', channel_multiplier=2)
    # elif enhance_mode == "Only Image Enhance":
    #     face_enhancer = None
    # else:
    #     face_enhancer = GFPGANer(model_path='GFPGANv1.4.pth', upscale=scale, arch='clean', channel_multiplier=2, bg_upsampler=upsampler)
    input_image = cv2.imread(input_image_path)
    img = cv2.cvtColor(np.array(input_image), cv2.COLOR_RGB2BGR)

    h, w = img.shape[0:2]
    if h < 300:
        img = cv2.resize(img, (w * 2, h * 2), interpolation=cv2.INTER_LANCZOS4)

    # if face_enhancer is not None:
    #     _, _, output = face_enhancer.enhance(img, has_aligned=False, only_center_face=only_face, paste_back=True)
    # else:
    output, _ = upsampler.enhance(img, outscale=scale)

    h, w = output.shape[0:2]
    max_size = 3480
    if h > max_size:
        w = int(w * max_size / h)
        h = max_size

    if w > max_size:
        h = int(h * max_size / w)
        w = max_size

    output = cv2.resize(output, (w, h), interpolation=cv2.INTER_LANCZOS4)

    enhanced_image = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)

    # # Generate unique filename with timestamp
    # filename = f"enhanced.png"
    # enhanced_path = os.path.join(output_dir, filename)

    # # Save the enhanced image
    # enhanced_image.save(enhanced_path, quality=100)

    # Display the image in Colab
    # from IPython.display import display
    #display(enhanced_image)

    #print(f"Enhanced image saved to: {enhanced_path}")

    return enhanced_image

In [ ]:
import cv2
import numpy as np
import os

def gamma_clahe(image_path, gamma=1.3):
    # Load image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Image not found or invalid image path")

    # Apply CLAHE
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    limg = cv2.merge((cl, a, b))
    clahe_img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

    # Apply Gamma Correction
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    final_img = cv2.LUT(clahe_img, table)

    return final_img

In [ ]:
def shadow_fight(image_path: str, bc_alpha: float = 1.4, bc_beta: int = 40):
    original_image = cv2.imread(image_path)
    enhanced_image = cv2.convertScaleAbs(original_image, alpha=bc_alpha, beta=bc_beta)

    return enhanced_image

In [ ]:
def grayscale(image_path):
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    return gray_image

In [ ]:
folder = "/content/drive/MyDrive/MV Sample"
targets = ["72399", "75513", "75845", "79084", "83013", "83622", "87060", "91894", "92043"]
for folder_name in os.listdir(folder):
    folder_path = os.path.join(folder, folder_name)
    if os.path.isdir(folder_path) and folder_name.isdigit():
        print(folder_path)
        try:
          if "passport_cropped.jpg" in os.listdir(folder_path):
            output_image = grayscale(os.path.join(folder_path, "passport_cropped.jpg"))
          else:
            output_image = grayscale(os.path.join(folder_path, "passport.jpg"))

          cv2.imwrite(os.path.join(folder_path, "passport_crop_gray.jpg"), output_image)
        except Exception as e:
          print(e)
          continue

/content/drive/MyDrive/MV Sample/89519
/content/drive/MyDrive/MV Sample/73470
/content/drive/MyDrive/MV Sample/74612
/content/drive/MyDrive/MV Sample/76250
/content/drive/MyDrive/MV Sample/89152
/content/drive/MyDrive/MV Sample/72360
/content/drive/MyDrive/MV Sample/94988
/content/drive/MyDrive/MV Sample/91737
/content/drive/MyDrive/MV Sample/63410
/content/drive/MyDrive/MV Sample/44414
/content/drive/MyDrive/MV Sample/72786
/content/drive/MyDrive/MV Sample/89839
/content/drive/MyDrive/MV Sample/89902
/content/drive/MyDrive/MV Sample/83515
/content/drive/MyDrive/MV Sample/75568
/content/drive/MyDrive/MV Sample/73992
/content/drive/MyDrive/MV Sample/74019
/content/drive/MyDrive/MV Sample/85193
/content/drive/MyDrive/MV Sample/74729
/content/drive/MyDrive/MV Sample/76816
/content/drive/MyDrive/MV Sample/74338
/content/drive/MyDrive/MV Sample/57299
/content/drive/MyDrive/MV Sample/76509
/content/drive/MyDrive/MV Sample/57357
/content/drive/MyDrive/MV Sample/41360
/content/drive/MyDrive/MV